# MNIST Convolutional Network

* 建立卷積神經網路 (convolutional neural network)
* 用 MNIST 來訓練 CNN
* 評估模型

In [1]:
!pip install tensorflow
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## 建立多層的卷積神經網路

前一個模型我們得到 92% 的準確率是非常不及格的．在這裡我們將建立一個更為適當且複雜的模型．也就是一個小型的 **卷積神經網路** (convolutional neural network)．在 MNIST 的準確度會提升到 99.2% 或許不是最好但也相當不錯的成績．

## 權重初始化

在建立模型之前我們需要建立一系列的權重 (weights) 還有偏移量 (biases)．為了避免 權重 (weight) 過於對稱還有 0 梯度 (gradient) 我們必須加入一小部分的為正的噪音 (noise)．因為我們使用的是 [ReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks))，因此特別適合在偏移量 (bias) 初始化為為正的數值來避免 **'神經元死去 (dead neurons)'**．這裡建立了兩個函數來避免每次使用他的時候都重新建立程式碼．

In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

## 卷積還有池化 (Convolution and Pooling)

(在這裡實在不知道怎麼翻譯，因此用阿陸仔的名詞來稱呼)

Tensorflow 同樣給我們很大的彈性來做卷積還有池化這兩個動作．如何處理邊界? 我們的 stride 大小要設多少? 在這個範例中，我們會一直使用 vanilla 的版本．我們的卷積過程中的參數 `stride` 會是 1 而 `padded` 則是 0．也因此輸入還有輸出都會是同樣的大小 (size)．而我們的 polling 則是用 2X2 的傳統 polling 來做．為了讓我們的程式更加簡潔，我們同樣把這樣的操作抽象成函數．

In [3]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding = 'SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding = 'SAME')

## 第一個卷積層

我們現在可以來實現第一個卷積層．他會先有一個卷積接著一個 max polling 來完成．這個卷積會從 5x5 的 patch 算出 32 個特徵．他的權重 tensor 的形狀是 [5, 5, 1, 32]．頭兩個維度是 patch 的大小，下一個則是輸入的 channels，最後一個則是輸出的 channels．同樣的在輸出也會有偏移量向量 (bias vector)．

In [4]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

現在要來把輸入 `x` 來導入我們剛剛建立的第一個卷積層，那必須先把 `x` 轉換成一個 4d 的 tensor，其中第二個和第三個維度對應到了圖片的寬度和高度，而最後一個則對應到了顏色的 channel 數 (這裡因為是灰階的所以 channel 為 1)

In [5]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

我們接下來把 `x_image` 還有權重 tensor 輸入剛剛定義的卷積函數，再來加上偏移值 (bias) 後輸入 **ReLU** 函數，最後經過 max pooling． `max_pool_2x2` 函數會把圖片的大小縮成 14x14．

In [6]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

## 第二個卷積層

為了建立比較深度的神經網路，我們把許多層疊在一起．第二層會從 5x5 的 patch 中取出 64 個特徵．

In [7]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

## 密集的全連接層

現在讓我們想像圖片的大小已經被縮成了 7x7 的大小，我們加入一個 1024 的全連接層來把前面的全部輸出輸入全連接層．其中包含了先把 pooling 的輸出展開後乘上一個權重矩陣再加上一個偏移量向量，最後輸入 ReLU．

In [8]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

## Dropout

為了減少 overfitting，在輸出層之前我們會加入 dropout 層．首先我們建立了一個站位子 (`placeholder`) 來代表神經元在 dropout 過程中不變的機率．這可以讓我們決定要在訓練的時候打開 dropout，而在測試的時候關閉 dropout． Tensorflow 的 `tf.nn.dropout` 除了會遮蔽神經元的輸出以外也會自動對輸入值做 scaling，所以我們在使用的時候可以不用考慮 scale．

註: 對於這種比較小的卷積網路，有沒有 dropout 對於成果不會有太大影響．dropout 是一個非常好的方法來減低 overfitting，但僅限於比較大的神經網路．

In [9]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## 輸出層

最後我們加上像之前 softmax regression 一樣的層．

In [10]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

## 訓練以及評估模型

那我們的模型表現的如何呢? 這裡我們會用之前 `Softmax` 範例的大部分程式碼來訓練以及評估這個模型．

不過有幾點不同的是:

* 我們會把 gradient descent 最佳化演算法換成更為精密的 ADAM 最佳化演算法．
* 我們會在 `feed_dict` 參數之中加入 `keep_prob` 這個參數來控制 dropout 的機率．
* 我們會在每一百個回合的訓練中印出紀錄

現在你可以來執行這段程式，但它需要 20,000 回合的訓練，可能會需要比較久的時間 (大概半小時)，當然如果你的處理器比較好的話也可能會比較快．

In [11]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.1
step 100, training accuracy 0.78
step 200, training accuracy 0.94
step 300, training accuracy 0.84
step 400, training accuracy 0.98
step 500, training accuracy 0.94
step 600, training accuracy 1
step 700, training accuracy 0.96
step 800, training accuracy 0.92
step 900, training accuracy 1
step 1000, training accuracy 0.98
step 1100, training accuracy 0.96
step 1200, training accuracy 0.98
step 1300, training accuracy 0.96
step 1400, training accuracy 1
step 1500, training accuracy 0.9
step 1600, training accuracy 1
step 1700, training accuracy 0.96
step 1800, training accuracy 1
step 1900, training accuracy 0.96
step 2000, training accuracy 1
step 2100, training accuracy 0.98
step 2200, training accuracy 1
step 2300, training accuracy 1
step 2400, training accuracy 1
step 2500, training accuracy 0.94
step 2600, training accuracy 0.96
step 2700, training accuracy 0.98
step 2800, training accuracy 1
step 2900, training accuracy 0.98
step 3000, training accu

最後的測試準確度大約為 99.2%

我們學到了用 Tensorflow 是可以很快速而且簡單的建置，訓練，以及評估一個較為精密的深度學習模型．